In [3]:
# -*- coding: utf-8 -*-
"""
InnerKurly Survey → Plotly Dashboard
- 입력: CSV (예: /mnt/data/미래전략기획실_고개리서치_2 - 시트1 (3).csv)
- 출력: 대시보드 HTML (/mnt/data/innerkurly_survey_dashboard.html)
"""

import re
import os
import pandas as pd
from collections import Counter, defaultdict
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# =========================
# CONFIG: 파일 경로 & 키워드
# =========================
CSV_PATH = "미래전략기획실_고개리서치_2 - 시트1 (3).csv"  # 환경에 맞게 수정
OUTPUT_HTML = "innerkurly_survey_dashboard.html"

# 컬럼 자동탐지 키워드(정규식 일부 허용)
KW = {
    "gender": ["성별", "gender"],
    "age": ["연령대", "연령", "나이", r"\bage\b"],
    "employment": ["직업", "직업군", "재직", "직장", "job", "employment"],

    # 분석 항목
    "lunch_pain": ["점심.*(아쉬|불만|문제)", "점심.*불편", "점심.*개선", "점심.*고민"],
    "diet_app_use": ["(다이어트|식단).*(앱|어플).*(사용|경험)", "식단.*앱.*이용"],
    "diet_app_pain": ["(다이어트|식단).*(앱|어플).*(불만|문제|어려움|불편)"],
    "diet_value": ["다이어트.*(가치|목표)", "중요.*가치관", "다이어트.*중요.*가치"],
    "ingredient_reason": ["(식자재|재료).*(구매처|구매).*(이유|선택)", "구매처.*선택.*이유"],
    "open_text": ["자유(응답|서술|기재)", "기타(의견|의사)", "의견", "코멘트", "코멘트", "서술", "설명", "하고싶은.*말"],

    # 선택(있으면 AB 비교)
    "ab_group": ["AB.?그룹", "A.?/?B.?그룹", r"\bab(_?group)?\b", "group", "그룹"],
    "pref_image": ["이미지.*선호", "사진.*선호", "배경.*선호", "컨텍스트.*선호"],
    "pref_copy": ["문구.*선호", "카피.*선호", "메시지.*선호"],
    "pref_offer": ["오퍼.*선호", "혜택.*선호", "프로모션.*선호"],
}

# 타겟 필터 값(가능한 값들 포함)
VAL = {
    "female": ["여", "여성", "여자", "female", "f", "FEMALE", "F"],
    "employed_hint": ["재직", "직장인", "회사원", "full", "상근", "근로", "employed", "working"],
}

# 오픈엔드 키워드(간단 카운트용)
OPEN_KW = ["가격", "가성비", "비싸", "저렴", "할인", "쿠폰",
           "다양", "질림", "메뉴", "맛", "신선",
           "편리", "편의", "귀찮", "간편", "앱", "정확",
           "배송", "정시", "도착", "보냉",
           "신뢰", "후기", "리뷰",
           "건강", "단백질", "샐러드"]


# =========================
# 유틸 함수
# =========================
def find_cols(df: pd.DataFrame, key_list):
    """
    주어진 키워드(리스트)를 이용해 df 컬럼 중 매칭되는 것들을 반환.
    정규식 / 부분일치 모두 지원.
    """
    cols = []
    for c in df.columns:
        c_norm = str(c).strip()
        for kw in key_list:
            pattern = kw
            if not kw.startswith("(?i)"):
                pattern = "(?i)" + kw
            if re.search(pattern, c_norm, flags=re.IGNORECASE):
                cols.append(c)
                break
    return list(dict.fromkeys(cols))  # unique & keep order


def pick_first(df, keys):
    cols = find_cols(df, keys)
    return cols[0] if cols else None


def normalize_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip()


def split_multi_value(cell):
    """
    체크박스/복수선택 항목을 쉼표/슬래시/세미콜론/파이프로 분리
    """
    if pd.isna(cell):
        return []
    s = str(cell)
    parts = re.split(r"[\/\|;, ]+", s)
    parts = [p.strip() for p in parts if p.strip()]
    return parts


def count_multiselect(series: pd.Series):
    cnt = Counter()
    for v in series.dropna():
        for item in split_multi_value(v):
            cnt[item] += 1
    return cnt


def count_categorical(series: pd.Series):
    return series.dropna().astype(str).str.strip().value_counts()


def approx_is_female(v):
    v = normalize_str(v)
    return v in VAL["female"] or v.lower() in [s.lower() for s in VAL["female"]]


def approx_is_30s(v):
    s = normalize_str(v)
    if not s:
        return False
    # '30대' 포함 or 숫자 범위 30~39
    if "30" in s:
        return True
    # 숫자 추출
    nums = re.findall(r"\d+", s)
    if nums:
        try:
            n = int(nums[0])
            return 30 <= n <= 39
        except:
            return False
    return False


def approx_is_employed(v):
    s = normalize_str(v)
    if not s:
        return False
    low = s.lower()
    if any(k in s for k in VAL["employed_hint"]):
        return True
    if any(k in low for k in [kk.lower() for kk in VAL["employed_hint"]]):
        return True
    return False


def warn(msg):
    print(f"[경고] {msg}")


# =========================
# 데이터 로드
# =========================
df = pd.read_csv(CSV_PATH, encoding="utf-8", engine="python")
print(f"[INFO] Rows: {len(df):,}  Cols: {len(df.columns)}")
print("[INFO] Columns:", list(df.columns))

# =========================
# 타겟 세그먼트 필터(30대·여성·재직)
# =========================
col_gender = pick_first(df, KW["gender"])
col_age = pick_first(df, KW["age"])
col_emp = pick_first(df, KW["employment"])

mask_all = pd.Series([True] * len(df))

if col_gender:
    mask_gender = df[col_gender].apply(approx_is_female)
    mask_all &= mask_gender
else:
    warn("성별 컬럼을 찾지 못했습니다. (전체 대상으로 진행)")

if col_age:
    mask_age = df[col_age].apply(approx_is_30s)
    mask_all &= mask_age
else:
    warn("연령(나이) 컬럼을 찾지 못했습니다. (성별만으로 진행 또는 전체)")

if col_emp:
    mask_emp = df[col_emp].apply(approx_is_employed)
    # 고용 컬럼이 애매하면 지나치게 제한하지 않도록 선택적으로만 적용
    if mask_emp.sum() / max(len(df), 1) >= 0.2:  # 20% 이상 매칭되면 적용
        mask_all &= mask_emp
    else:
        warn("재직/직업 컬럼은 감지했으나 매칭률이 낮아 필터에 강제 적용하지 않았습니다.")
else:
    warn("재직/직업 컬럼을 찾지 못했습니다. (성별/연령만으로 진행)")

df_tgt = df[mask_all].copy()
print(f"[INFO] 타겟(30대·여성·재직 추정) 샘플 수: {len(df_tgt):,}")

# =========================
# 시각화 생성
# =========================
figs = []
captions = []

def add_fig(fig, caption):
    figs.append(fig)
    captions.append(caption)

# 1) 점심 불편(타겟)
col_lunch = pick_first(df, KW["lunch_pain"])
if col_lunch:
    cnt = count_multiselect(df_tgt[col_lunch]) if df_tgt[col_lunch].dtype == object else count_categorical(df_tgt[col_lunch])
    s = pd.Series(cnt).sort_values(ascending=False)
    if len(s) > 0:
        fig = px.bar(s.reset_index(), x='index', y=0, title="타겟(30대 여성) 점심 관련 불편 TOP", labels={'index': '불편 항목', '0': '응답 수'})
        fig.update_layout(xaxis_tickangle=-30)
        add_fig(fig, "점심 관련 불편(타겟)")
else:
    warn("점심 불편 관련 컬럼을 찾지 못했습니다. (해당 차트 건너뜀)")

# 2) 식단/다이어트 앱 사용 경험 & 불만
col_app_use = pick_first(df, KW["diet_app_use"])
if col_app_use:
    s = count_categorical(df_tgt[col_app_use])
    if len(s) > 0:
        fig = px.bar(s.reset_index(), x='index', y=col_app_use, title="타겟의 식단/다이어트 앱 사용 경험", labels={'index': '응답', col_app_use: '응답 수'})
        fig.update_layout(xaxis_tickangle=-30)
        add_fig(fig, "식단/다이어트 앱 사용 경험(타겟)")
else:
    warn("식단/다이어트 앱 사용 경험 컬럼을 찾지 못했습니다.")

col_app_pain = pick_first(df, KW["diet_app_pain"])
if col_app_pain:
    cnt = count_multiselect(df_tgt[col_app_pain]) if df_tgt[col_app_pain].dtype == object else count_categorical(df_tgt[col_app_pain])
    s = pd.Series(cnt).sort_values(ascending=False)
    if len(s) > 0:
        fig = px.bar(s.reset_index(), x='index', y=0, title="타겟의 식단/다이어트 앱 불만 TOP", labels={'index': '불만 항목', '0': '응답 수'})
        fig.update_layout(xaxis_tickangle=-30)
        add_fig(fig, "식단/다이어트 앱 불만(타겟)")
else:
    warn("식단/다이어트 앱 불만 컬럼을 찾지 못했습니다.")

# 3) 다이어트에서 가장 중요한 가치
col_value = pick_first(df, KW["diet_value"])
if col_value:
    cnt = count_multiselect(df[col_value]) if df[col_value].dtype == object else count_categorical(df[col_value])
    s = pd.Series(cnt).sort_values(ascending=False)
    if len(s) > 0:
        fig = px.pie(s.reset_index(), names='index', values=0, title="다이어트에서 가장 중요한 가치(전체)")
        add_fig(fig, "다이어트 가치(전체)")
else:
    warn("다이어트 가치/목표 관련 컬럼을 찾지 못했습니다.")

# 4) 식자재 구매처/선택 이유
col_ing = pick_first(df, KW["ingredient_reason"])
if col_ing:
    cnt = count_multiselect(df[col_ing]) if df[col_ing].dtype == object else count_categorical(df[col_ing])
    s = pd.Series(cnt).sort_values(ascending=False)
    if len(s) > 0:
        fig = px.bar(s.reset_index(), x='index', y=0, title="식자재 구매처 선택 이유(전체)", labels={'index': '이유', '0': '응답 수'})
        fig.update_layout(xaxis_tickangle=-30)
        add_fig(fig, "구매처 선택 이유(전체)")
else:
    warn("식자재 구매처 선택 이유 컬럼을 찾지 못했습니다.")

# 5) (선택) AB/선호도 비교 — 컬럼 있을 때만
col_ab = pick_first(df, KW["ab_group"])
col_pref_img = pick_first(df, KW["pref_image"])
col_pref_copy = pick_first(df, KW["pref_copy"])
col_pref_offer = pick_first(df, KW["pref_offer"])

def add_pref_plot(col_pref, label):
    s = None
    if not col_pref:
        return
    if col_ab:
        grp = df.groupby([col_ab, col_pref]).size().reset_index(name="cnt")
        fig = px.bar(grp, x=col_pref, y="cnt", color=col_ab, barmode="group",
                     title=f"{label} 선호도(AB 그룹별)", labels={col_pref: label, "cnt": "응답 수", col_ab: "그룹"})
        fig.update_layout(xaxis_tickangle=-30)
    else:
        s = count_categorical(df[col_pref])
        fig = px.bar(s.reset_index(), x='index', y=col_pref, title=f"{label} 선호도(전체)", labels={'index': label, col_pref: '응답 수'})
        fig.update_layout(xaxis_tickangle=-30)
    add_fig(fig, f"{label} 선호도")

add_pref_plot(col_pref_img, "이미지/배경")
add_pref_plot(col_pref_copy, "문구/메시지")
add_pref_plot(col_pref_offer, "오퍼/혜택")

# 6) 자유응답 키워드 카운트 (간단)
open_cols = find_cols(df, KW["open_text"])
if open_cols:
    text_all = []
    for c in open_cols:
        text_all.extend(df[c].dropna().astype(str).tolist())
    text_blob = "\n".join(text_all)
    kw_cnt = []
    for kw in OPEN_KW:
        # '가격'과 같이 짧은 단어는 부분일치 허용 (대소문자 무시)
        n = len(re.findall(kw, text_blob, flags=re.IGNORECASE))
        if n > 0:
            kw_cnt.append((kw, n))
    kw_cnt.sort(key=lambda x: x[1], reverse=True)
    if kw_cnt:
        s = pd.DataFrame(kw_cnt, columns=["키워드", "언급수"])
        fig = px.bar(s, x="키워드", y="언급수", title="자유응답 키워드 TOP", labels={"키워드": "키워드", "언급수": "언급 수"})
        fig.update_layout(xaxis_tickangle=-30)
        add_fig(fig, "자유응답 키워드")
else:
    warn("자유응답/의견 컬럼을 찾지 못했습니다.")

# =========================
# HTML 대시보드 저장
# =========================
html_parts = []
html_parts.append("<html><head><meta charset='utf-8'><title>InnerKurly Survey Dashboard</title></head><body>")
html_parts.append("<h1>InnerKurly 설문 대시보드</h1>")

for i, (fig, cap) in enumerate(zip(figs, captions), start=1):
    html_parts.append(f"<h2>{i}. {cap}</h2>")
    html_parts.append(fig.to_html(full_html=False, include_plotlyjs='cdn'))

html_parts.append("</body></html>")
html = "\n".join(html_parts)

with open(OUTPUT_HTML, "w", encoding="utf-8") as f:
    f.write(html)

print(f"[완료] 대시보드 저장: {OUTPUT_HTML}")
print("[Tip] 특정 컬럼을 못 찾으면 CONFIG 상단 KW 사전에 키워드를 추가해 주세요.")


[INFO] Rows: 93  Cols: 32
[INFO] Columns: ['날짜', '이름', '이메일', '성별', '연령대', '직업', '점심_해결방식', '점심_정보출처', '점심_식대', '점심_아쉬운점', '저녁_주말식사', '식재료_구매처', '식재료_정보출처', '식재료_구매빈도', '식재료_지출금액', '식재료_구매이유', '영양제_섭취빈도', '영양제_종류', '영양제_정보출처', '영양제_구매처', '영양제_구매빈도', '영양제_지출금액', '영양제_구매이유', '다이어트_경험', '다이어트_정보출처', '다이어트_방식', '다이어트앱_아쉬운점', '다이어트_가치', '자유의견', '개인정보동의', '메일 내용', '전송 상태']
[경고] 재직/직업 컬럼은 감지했으나 매칭률이 낮아 필터에 강제 적용하지 않았습니다.
[INFO] 타겟(30대·여성·재직 추정) 샘플 수: 34
[경고] 식단/다이어트 앱 사용 경험 컬럼을 찾지 못했습니다.
[경고] 식단/다이어트 앱 불만 컬럼을 찾지 못했습니다.
[완료] 대시보드 저장: innerkurly_survey_dashboard.html
[Tip] 특정 컬럼을 못 찾으면 CONFIG 상단 KW 사전에 키워드를 추가해 주세요.
